Берем модель для суммаризации текстов: https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

In [1]:
import pandas as pd
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

И запускаем суммаризацию текстов на тестовой части вот этого датасета: https://huggingface.co/datasets/IlyaGusev/gazeta.

In [2]:
from datasets import load_dataset
import evaluate
dataset = load_dataset("IlyaGusev/gazeta")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
})

In [3]:
df = pd.DataFrame(dataset['test'])
df.head()

,text,summary,title,date,url
0,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Германии объяснили упоминание имени Путина н...,2020-09-01 00:22:59,https://www.gazeta.ru/politics/2020/08/31_a_13...
1,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Делегации Израиля и США прибыли в ОАЭ для обсу...,2020-09-01 08:08:16,https://www.gazeta.ru/politics/2020/08/31_a_13...
2,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Оппозиция Белоруссии объявила о создании новой...,2020-09-01 09:21:38,https://www.gazeta.ru/politics/2020/09/01_a_13...
3,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает крайне опасными действия США на...,2020-09-01 09:33:30,https://www.gazeta.ru/army/2020/09/01/13222904...
4,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,В России вступил в силу закон о внесудебном ба...,2020-09-01 09:49:24,https://www.gazeta.ru/business/2020/09/01/1322...


In [4]:
import torch
device = torch.device('cuda:0')
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

Параллельно считаем метрики

In [5]:
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [6]:
from tqdm import tqdm

bleu_scores = []
rouge_scores = []
for text in tqdm(dataset['test']['text'], total=len(dataset['test']['text'])):

    input_ids = tokenizer(
        text,
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]
    input_ids = input_ids.to(device)
    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    bleu_scores.append(bleu.compute(predictions=[summary], references=[text]))
    rouge_scores.append(rouge.compute(predictions=[summary], references=[text]))

100%|██████████| 6793/6793 [3:47:20<00:00,  2.01s/it]  


Средние значения метрик

In [21]:
average_bleu = sum([score['bleu'] for score in bleu_scores]) / len(bleu_scores)
average_rouge = {}
for metric in rouge_scores[0].keys():
    average_rouge[metric] = sum([score[metric] for score in rouge_scores]) / len(rouge_scores)
print(f'Средняя BLEU-оценка: {average_bleu:.2}')
for k in average_rouge:
    average_rouge[k] = round(average_rouge[k], 3)
print(f'Средняя ROUGE-оценка: {average_rouge}')

Средняя BLEU-оценка: 0.00048
Средняя ROUGE-оценка: {'rouge1': 0.139, 'rouge2': 0.07, 'rougeL': 0.137, 'rougeLsum': 0.137}
